# Table of Contents
* [1. Import Libraries](#Part_1)
* [2. Read Data](#Part_2)
* [3. Linearization](#Part_3)
* [4. Initialize the model and Define sets](#Part_4)
* [5. Define variables](#Part_5)
* [6. Objective function](#Part_6)
* [7. Constraints](#Part_7)
    * [7.1. Start up & Shut down](#Part_7_1)
    * [7.2. Min up time & Min down time](#Part_7_2)
    * [7.3. Lower and upper bounds of thermal power generation](#Part_7_3)
    * [7.4. Ramp up & Ramp down](#Part_7_4)
    * [7.5. Excess electricity limit & Curtailment limit](#Part_7_5)
    * [7.6. Balance (Demand satisfaction)](#Part_7_6)
    * [7.7. Sum of piece wise powers](#Part_7_7)
    * [7.8. Emission constraints](#Part_7_8)
    * [7.9. Transmission constraints](#Part_7_9)
    * [7.10. Storage Constraints](#Part_7_10)
    * [7.11. N-1 Contingency](#Part_7_11)
* [8. Solve the model](#Part_8)
    * [8.1. Save the results in an excel file](#Part_8_1)
    * [8.2. Plot results and add them to the excel file](#Part_8_2)
* [9. To do list](#Part_9)

# 1. Import libraries <a class="anchor" id="Part_1"></a>

In [1]:
from pyomo.core.expr.numeric_expr import LinearExpression
from pyomo.opt import SolverStatus, TerminationCondition
import matplotlib.pyplot as plt
import pyomo.environ as pyo
import pandas as pd
import numpy as np
import openpyxl
import requests
import warnings
import time
import json
import os

plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams["font.size"] = 8

warnings.filterwarnings('ignore')

# 2. Read Data <a class="anchor" id="Part_2"></a>

In [2]:
address= "path_to_excel_file/"

#################################### General parameters #########################################################

Params = pd.read_excel(address+"Data.xlsx",header= 0,index_col= 0,sheet_name= "Params", usecols="A:B")

#################################### power and reserve market price #############################################

Market = pd.read_excel(address+"Data.xlsx",header= 0,index_col= 0,sheet_name= "Market")

#################################### Generator parameters #######################################################

GenData = pd.read_excel(address+"Data.xlsx",header= 0,index_col= 0,sheet_name= "Generators")
GenNode = pd.read_excel(address+"Data.xlsx",header= 0,index_col= 0,sheet_name= "GenNodes")
G       = list(GenData.index) # will be used as the set of generators in the optimization model
NumGen  = len(G)

#################################### Transmission parameters ####################################################

TransEff  = pd.read_excel(address+"Data.xlsx",header= 0,index_col= 0,sheet_name= "TransEffs")
TransUb   = pd.read_excel(address+"Data.xlsx",header=0,index_col=0,sheet_name="TransUb")
TransCost = pd.read_excel(address+"Data.xlsx",header=0,index_col=0,sheet_name="TransCost")

##################################### Electricity Load ##########################################################

load = pd.read_excel(address+"Data.xlsx",header= 0,index_col= 0,sheet_name= "Loads")
T    = list(load.index)   # will be used as the set of time in the optimization model
N    = list(load.columns) # will be used as the set of nodes in the optimization model
Horizon = len(T)

##################################### Renewables data ###########################################################

if Params.loc['consider_Renewable','Val'] == True:

    RenNode = pd.read_excel(address+"Data.xlsx",header= 0,index_col= 0,sheet_name= "RenNode")
    R       = list(RenNode.columns.values)

    if Params.loc['use_ninja','Val'] == True:

        token = Params.loc['ninja_token','Val']
        api_base = 'https://www.renewables.ninja/api/'
        ses = requests.session()
        ses.headers = {'Authorization': 'Token ' + token}

        RenGenNinja  = pd.read_excel(address+"Data.xlsx",header= 0,index_col= 0,sheet_name= "RenGenNinja",usecols="B:XFD")

        RenGen = np.zeros((Horizon,len(R)))

        for r in R:
            temp = RenGenNinja.loc['type',r]
            url = api_base + f'data/{temp}'

            if RenGenNinja.loc['type',r] == 'pv':

                if RenGenNinja.loc['tracking',r] == 'No':
                    temp_tracking = 0
                elif RenGenNinja.loc['tracking',r] == 'single (azimuth)':
                    temp_tracking = 1
                else: # RenGenNinja.loc['tracking',r] = 'dual (azimuth & tilt)'
                    temp_tracking = 2

                args = {
                'lat'        : RenGenNinja.loc['lat',r],
                'lon'        : RenGenNinja.loc['lon',r],
                'date_from'  : RenGenNinja.loc['date_from','r1'][1:-1],
                'date_to'    : RenGenNinja.loc['date_to','r1'][1:-1],
                'local_time' : RenGenNinja.loc['local_time',r],
                'dataset'    : RenGenNinja.loc['dataset',r],
                'capacity'   : RenGenNinja.loc['capacity',r],
                'system_loss': RenGenNinja.loc['system_loss',r],
                'tracking'   : temp_tracking,
                'tilt'       : RenGenNinja.loc['tilt',r],
                'azim'       : RenGenNinja.loc['azim',r],
                'interpolate': RenGenNinja.loc['interpolate',r],
                'format'     : RenGenNinja.loc['format',r]            }
                request = ses.get(url, params=args)

            else: # type = wind
                args = {
                'lat'        : RenGenNinja.loc['lat',r],
                'lon'        : RenGenNinja.loc['lon',r],
                'date_from'  : RenGenNinja.loc['date_from','r1'][1:-1],
                'date_to'    : RenGenNinja.loc['date_to','r1'][1:-1],
                'local_time' : RenGenNinja.loc['local_time',r],
                'dataset'    : RenGenNinja.loc['dataset',r],
                'capacity'   : RenGenNinja.loc['capacity',r],
                'height'     : RenGenNinja.loc['height',r],
                'turbine'    : RenGenNinja.loc['turbine',r],
                'interpolate': RenGenNinja.loc['interpolate',r],
                'format'     : RenGenNinja.loc['format',r]            }
                request = ses.get(url, params=args)


            if request.ok == False:
                print(f"{r}:{RenGenNinja.loc['type',r]}, Request failed.")
            else:
                print(f"{r}:{RenGenNinja.loc['type',r]}, Successful request!")


            parsed_response = json.loads(request.text)
            data = pd.read_json(json.dumps(parsed_response['data']), orient='index')


            UTC_diff = str(data.iloc[0,0]).split('+')[1].split(':')
            if float(UTC_diff[1]) >= 30:
                round_hour = int(UTC_diff[0]) + 1
            else:
                round_hour = int(UTC_diff[0])


            data = np.array(data.loc[:,'electricity'])
            data = data[:-round_hour].copy()
            data = np.concatenate( ( np.zeros((1,round_hour))[0], data ) )
            RenGen[:,int(r[-1])-1] = data

            time.sleep(1)
        
        #divided by 1000 to turn KW to MW as the ninja output is in KW
        RenGen = pd.DataFrame(RenGen/1000, columns=R, index= T) 

    else: # do not use ninja, use my own data

        RenGen  = pd.read_excel(address+"Data.xlsx",header= 0,index_col= 0,sheet_name= "RenGen")

################################### Node spesific parameters ####################################################

NodeData = pd.read_excel(address+"Data.xlsx",header=0,index_col=0,sheet_name="NodeData")

################################### Storage parameters ###########################################################

if Params.loc['consider_Storage','Val'] == True:
    Storage   = pd.read_excel(address+"Data.xlsx",header=0,index_col=0,sheet_name="Storage")
    StorNodes = pd.read_excel(address+"Data.xlsx",header=0,index_col=0,sheet_name="StorNodes")
    S = list(Storage.index) # will be used as the set of storages in the optimization model

################################### Transmission outage #########################################################

TransOutage = pd.read_excel(address+"Data.xlsx",header=0,index_col=0,sheet_name="TransOutage")

headers = []
for n in N:
    for nn in N:
        if n != nn:
            headers.append(n+nn)

Trans_Outage = pd.DataFrame( np.ones(( len(T),len(N)*(len(N)-1) )), T, headers)
for n in N:
    for nn in N:
        if n != nn:
            if TransOutage.loc[n,nn] != 0:
                OutageParts = TransOutage.loc[n,nn].split(',')
                for part in OutageParts:
                    OneOutagePart = part.split('::')
                    if len(OneOutagePart) == 2:
                        Trans_Outage.loc[int(OneOutagePart[0]):int(OneOutagePart[1]),n+nn] = 0
                    else:
                        Trans_Outage.loc[int(OneOutagePart[0]),n+nn] = 0

Trans_Outage = Trans_Outage.astype(int)

# 3. Linearization <a class="anchor" id="Part_3"></a>

In [3]:
cost_coef = [list(GenData.loc[:,"c"]),
             list(GenData.loc[:,"b"]),
             list(GenData.loc[:,"a"])]

emission_coef = [list(GenData.loc[:,"alpha"]),
                 list(GenData.loc[:,"beta"]),
                 list(GenData.loc[:,"gamma"]),
                 list(GenData.loc[:,"eta"]),
                 list(GenData.loc[:,"sigma"])]

pmin = np.array(GenData.loc[:,"pmin"])
pmax = np.array(GenData.loc[:,"pmax"])

def GenCost(cost_coef,P):
    return cost_coef[2]*np.power(P, 2) + cost_coef[1]*P + cost_coef[0]

def GenEmission(emission_coef,P):
    return emission_coef[2]*np.power(P,2)+emission_coef[1]*P+emission_coef[0]+emission_coef[3]*np.exp(emission_coef[4]*P)

lines_num = int(Params.loc['lines_num','Val'])      # how many lines you want to fit on the quadratic cost function
points_num = lines_num + 1
points = np.linspace(pmin,pmax,num=points_num)
costs = GenCost(cost_coef,points).T
emissions = GenEmission(emission_coef,points).T
points = points.T

CostSlope = np.zeros((NumGen, lines_num))     # slope of each fitted line on cost functins
EmissionSlope = np.zeros((NumGen, lines_num)) # slope of each fitted line on emission functins
for i in range(NumGen):
    for j in range(points_num-1):
        CostSlope[i,j] = (costs[i,j+1] - costs[i,j]) / (points[i,j+1] - points[i,j])
        EmissionSlope[i,j] = (emissions[i,j+1] - emissions[i,j]) / (points[i,j+1] - points[i,j])

linear_range = []    # distance between two consecutive points (beginning and end of each line)
for i in range(NumGen):
    linear_range.append(points[i,1] - points[i,0])

linear_range = pd.DataFrame(linear_range, columns = ['linear_range'], index = G)
CostSlope = pd.DataFrame(CostSlope, columns = list(range(1,lines_num +1)), index = G)
EmissionSlope = pd.DataFrame(EmissionSlope, columns = list(range(1,lines_num +1)), index = G)

# 4. Initialize the model and Define sets <a class="anchor" id="Part_4"></a>

In [4]:
model = pyo.ConcreteModel(name="Unit Commitment")

# G   defined above (when reading generators' data)
# T   defined above (when reading load data)
# N   defined above (when reading load data)
# S   defined above (when reading storages' data) ==> if Params.loc['consider_Storage','Val'] == True 
# R   defined above (when reading renewables data) ==> if Params.loc['consider_Renewable','Val'] == True
L = list(range(1,lines_num +1)) # each of linear part in the generators operational cost function

# 5. Define variables <a class="anchor" id="Part_5"></a>

In [5]:
######################################## Different parts of the OF ###################################################

model.SUSDC  = pyo.Var(within=pyo.NonNegativeReals)  # start up and shutdown costs
model.SUSDEC = pyo.Var(within=pyo.NonNegativeReals)  # start up and shutdown emission costs
model.OC     = pyo.Var(within=pyo.NonNegativeReals)  # operational costs
model.OEC    = pyo.Var(within=pyo.NonNegativeReals)  # operational emission costs
model.EEC    = pyo.Var(within=pyo.NonNegativeReals)  # excess electricity cost
model.CEC    = pyo.Var(within=pyo.NonNegativeReals)  # curtailed electricity cost
model.TC     = pyo.Var(within=pyo.NonNegativeReals)  # transmission cost

model.PI     = pyo.Var(within=pyo.NonNegativeReals)  # income from the power market

model.RI     = pyo.Var(within=pyo.NonNegativeReals)  # reserve incentive income
if Params.loc['consider_N-1Contingency','Val'] == False:
    model.RI.fix(0)

################################ Power generation and its linear parts + reserve #####################################

model.pl = pyo.Var(G,T,L,N,within=pyo.NonNegativeReals) # power at each time step for each linear part and generator
model.p  = pyo.Var(G,T,N,  within=pyo.NonNegativeReals, initialize = 0) # power at time t for generator g in node n

if Params.loc['consider_N-1Contingency','Val'] == True:
    model.r  = pyo.Var(G,T,N,  within=pyo.NonNegativeReals)
    for g in G:
        for t in T:
            for n in N:
                if GenNode.loc[n,g] != 1:
                    model.r[g,t,n].fix(0)

################################ power flow between nodes ##########################################################

model.f  = pyo.Var(T,N,N,within=pyo.NonNegativeReals) # energy flow from one node to another
for t in T:
    for n in N:
        for nn in N:
            if n == nn:                                     # one cannot send energy from n to n (itself)
                model.f[t,n,nn].fix(0) 
            if n != nn and Trans_Outage.loc[t,n+nn] == 0:   # at time t, the line from n to nn is out
                model.f[t,n,nn].fix(0)

model.fb  = pyo.Var(T,N,N,within=pyo.Binary)
################################ Excess and curtailed power #####################################################

model.ren_consumption = pyo.Var(T,N, within=pyo.NonNegativeReals)
model.ren_excess      = pyo.Var(T,N, within=pyo.NonNegativeReals)
if Params.loc['consider_Renewable','Val'] == False:
    for t in T:
        for n in N:
            model.ren_consumption[t,n].fix(0)
            model.ren_excess[t,n].fix(0)

model.thermal_consumption = pyo.Var(T,N, within=pyo.NonNegativeReals)
model.thermal_excess      = pyo.Var(T,N, within=pyo.NonNegativeReals)

model.EE = pyo.Var(T,N, within=pyo.NonNegativeReals) # Excess electricity at time t node n
model.CE = pyo.Var(T,N, within=pyo.NonNegativeReals) # curtailed electricity at time node n

################################ Demand response ################################################################

if Params.loc['consider_DemandResponse','Val'] == True:
    def demand_response_bounds(model,t,n):
        return ( load.loc[t,n]*(1-NodeData.loc[n,'DRPlb']), load.loc[t,n]*(1+NodeData.loc[n,'DRPub']) )
    model.D  = pyo.Var( T,N, within=pyo.NonNegativeReals, bounds=demand_response_bounds )

################################ Emission ######################################################################

model.emsn  = pyo.Var(G,T,N,   within=pyo.NonNegativeReals) # emission for generator g at time t and node n

################################ Storage variables #############################################################

if Params.loc['consider_Storage','Val'] == True:
    model.SLE  = pyo.Var(T,N,S, within=pyo.NonNegativeReals) # level of energy in storage s in node n at time t
    model.SCH  = pyo.Var(T,N,S, within=pyo.NonNegativeReals) # charging energy in storage s in node n at time t
    model.SDIS = pyo.Var(T,N,S, within=pyo.NonNegativeReals) # discharging energy in storage s in node n at time t
    model.SCHB = pyo.Var(T,N,S, within=pyo.Binary)
    model.SDISB= pyo.Var(T,N,S, within=pyo.Binary)
    for t in T:
        for n in N:
            for s in S:
                #if storage s does not exist in node n, SLE, SCH, and SDIS must be zero for s in n
                if StorNodes.loc[n,s] == 0:  
                    model.SLE[t,n,s].fix(0)   
                    model.SCH[t,n,s].fix(0)
                    model.SDIS[t,n,s].fix(0)
                    model.SCHB[t,n,s].fix(0)
                    model.SDISB[t,n,s].fix(0)
                else:
                    if t == Horizon:
                        model.SLE[t,n,s].fix(Storage.loc[s,'FinalEnergy'])

########################## Start up and shutdown variables #######################################################

model.v  = pyo.Var(G,T,N,  within=pyo.Binary, initialize = 0)   # turn gen g on at time t node n
model.w  = pyo.Var(G,T,N,  within=pyo.Binary, initialize = 0)   # turn gen g off at time t node n

########################### operation variables ##################################################################

model.u  = pyo.Var(G,T,N,  within=pyo.Binary, initialize = 0)   # whether gen g is working or not in time t node n
for g in G:
    for t in T:
        for n in N:
            if GenNode.loc[n,g] == 0:
                model.u[g,t,n].fix(0)  # if generator g is not in node n, u should be 0 at each t for gen g in n

                
model.a = pyo.Var(G,T,N,  within=pyo.Binary)   # (u=0) a = 1: r = 0,  a = 0, r = pmin
                                               # it says, if the gen is not working (u=0) then the offered reserve
                                               # could be either 0 or pmin

# 6. Objective function <a class="anchor" id="Part_6"></a>

In [6]:
def SUSD_Cost_rule(model): # start up and shut down costs
    return model.SUSDC == sum( GenData.loc[g,'StartUpCost']*model.v[g,t,n] + GenData.loc[g,'ShutDownCost']*model.w[g,t,n]
               for g in G for t in T for n in N )
model.SUSD_Cost = pyo.Constraint(rule=SUSD_Cost_rule)

#################################################################################################################

def SUSD_Emission_Cost_rule(model): # start up and shut down emission costs
    return model.SUSDEC == sum( NodeData.loc[n,'EmissionCost']*(GenData.loc[g,'StartUpEmission']*model.v[g,t,n]+\
                 GenData.loc[g,'ShutDownEmission']*model.w[g,t,n]) for n in N for g in G for t in T )
model.SUSD_Emission_Cost = pyo.Constraint(rule=SUSD_Emission_Cost_rule)

#################################################################################################################

def Operation_Cost_rule(model):
    fixed_OC    = sum( GenData.loc[g,'MinCost']*model.u[g,t,n] for g in G for t in T for n in N )
    variable_OC = sum( CostSlope.loc[g,l]*model.pl[g,t,l,n] for g in G for t in T for l in L for n in N )
    return model.OC == fixed_OC + variable_OC
model.Operation_Cost = pyo.Constraint(rule=Operation_Cost_rule)

#################################################################################################################

def Operation_emission_cost_rule(model):
    return model.OEC == sum (NodeData.loc[n,'EmissionCost']*model.emsn[g,t,n] for g in G for t in T for n in N )
model.Operation_emission_cost = pyo.Constraint(rule=Operation_emission_cost_rule)

#################################################################################################################

def Excess_electricity_cost_rule(model):
    return model.EEC == sum( NodeData.loc[n,'excess_ele_cost']*model.EE[t,n] for t in T for n in N )
model.Excess_electricity_cost = pyo.Constraint(rule=Excess_electricity_cost_rule)

#################################################################################################################

def Curtailed_electricity_cost_rule(model):
    return model.CEC == sum( NodeData.loc[n,'curtailed_ele_cost']*model.CE[t,n] for t in T for n in N )
model.Curtailed_electricity_cost = pyo.Constraint(rule=Curtailed_electricity_cost_rule)

#################################################################################################################

def Transmission_cost_rule(model):
    return model.TC == sum( TransCost.loc[n,nn]*model.f[t,n,nn] for t in T for n in N for nn in N)
model.Transmission_cost = pyo.Constraint(rule=Transmission_cost_rule)

#################################################################################################################

if Params.loc['consider_N-1Contingency','Val'] == True:
    def Reserve_market_rule(model):
        return model.RI == sum( Market.loc[t,'ReservePrice']*model.r[g,t,n] for g in G for t in T for n in N )
    model.Reserve_market = pyo.Constraint(rule=Reserve_market_rule)

#################################################################################################################
    
def Power_market_rule(model):
    if Params.loc['consider_Storage','Val'] == True:
        generation_income = sum(Market.loc[t,'PowerPrice']*(model.thermal_consumption[t,n]\
                                                           +model.ren_consumption[t,n]) for t in T for n in N)
        discharge_income = sum(Market.loc[t,'PowerPrice']*model.SDIS[t,n,s] for t in T for n in N for s in S)
        
        return model.PI == generation_income + discharge_income
        
    else:
        return model.PI == sum( Market.loc[t,'PowerPrice']*(model.thermal_consumption[t,n]+model.ren_consumption[t,n])
                                for t in T for n in N )
model.Power_market = pyo.Constraint(rule=Power_market_rule)

#################################################################################################################

def obj_rule(model):
    return Params.loc['cost_weight','Val']*(model.SUSDC+model.OC+model.EEC+model.CEC+model.TC-model.RI-model.PI) +\
           Params.loc['emission_weight','Val']*(model.SUSDEC+model.OEC)

model.obj = pyo.Objective(rule=obj_rule, sense = pyo.minimize)

# 7. Constraints <a class="anchor" id="Part_7"></a>

## 7.1. Start up & Shut down <a class="anchor" id="Part_7_1"></a>

In [7]:
def SU_rule(model,g,t,n):
    if t > 1:
        """after initiation, if the gen was not in use (ut-1=0) and now you want to use it (ut=1), then vt=1
        note that due to the startup cost the model tries to force v to be zero unless it is necessary to 
        turn the gen on"""
        return model.u[g,t,n] - model.u[g,t-1,n] <= model.v[g,t,n]
    else:
        #at the beginning if you want to use the generator (u=1), you should start it (v=1)
        return model.u[g,t,n] <= model.v[g,t,n]
model.SU = pyo.Constraint(G,T,N, rule = SU_rule)

def SD_rule(model,g,t,n):
    if t > 1:
        """after initiation, if the gen was in use (ut-1=1) and now you want to turn it off (ut=0), then wt=1.
        note that due to the shutdown cost the model tries to force w to be zero unless it is necessary to turn
        the gen off"""
        return model.u[g,t-1,n] - model.u[g,t,n] <= model.w[g,t,n]
    else:
        # since we initialized u with zero, we can not shut down at the beginning as there is nothing to shut it down
        return pyo.Constraint.Skip
model.SD = pyo.Constraint(G,T,N, rule = SD_rule)

## 7.2. Min up time & Min down time <a class="anchor" id="Part_7_2"></a>

In [8]:
def min_up_time_rule(model,g,t,n):
    """check if minup time exceeds the horizon or not, 
    pick the nearest one between the horizon and t+minuptime-1. 
    -1 is due the fact that time t itself is taken into account."""
    alpha = min(Horizon,t+GenData.loc[g,'MinOnTime']-1)
    
    """make a new time set from t to alpha. +1 is for considering alpha itself."""
    TT = list(range(t,alpha +1))
    
    """force the model to set u=1 during TT, if vt=1"""
    return sum( model.u[g,tt,n] for tt in TT ) >= len(TT)*model.v[g,t,n]
model.MUT = pyo.Constraint(G,T,N, rule = min_up_time_rule)

def min_down_time_rule(model,g,t,n):
    """check if mindown time exceeds the horizon or not, 
    pick the nearest one between the horizon and t+mindowntime-1. 
    -1 is due the fact that time t itself is taken into account."""
    alpha = min(Horizon,t+GenData.loc[g,'MinOffTime']-1)
    
    """make a new time set from t to alpha. +1 is for considering alpha itself."""
    TT = list(range(t,alpha +1))
    
    """force the model to set u=0 during TT, if wt=1"""
    return sum( 1-model.u[g,tt,n] for tt in TT ) >= len(TT)*model.w[g,t,n]
model.MDT = pyo.Constraint(G,T,N, rule = min_down_time_rule)

## 7.4. Ramp up & Ramp down <a class="anchor" id="Part_7_4"></a>

In [9]:
def RU_rule(model,g,t,n):
    if t > 1:
        """after initialization, you can increase gen output considering ramp up rate (first RHS term).
        if the gen wants to be started, the power will increase from 0 to *pmin* again (second RHS term)."""
        return model.p[g,t,n] - model.p[g,t-1,n] <= GenData.loc[g,'RampUp']*(1+model.u[g,t-1,n]-model.u[g,t,n]) +\
               GenData.loc[g,'pmin']*(2-model.u[g,t,n]-model.u[g,t-1,n])
    else:
        """at the beginning, if you turn generator on (v=u=1), then the gen output will be pmin"""
        return model.p[g,t,n] == model.u[g,t,n]*GenData.loc[g,'pmin']
model.RU = pyo.Constraint(G,T,N, rule = RU_rule)

def RD_rule(model,g,t,n):
    if t > 1:
        """after initialization, you can decrease gen output considering ramp down rate (first RHS term).
        if the gen wants to be shuted down, the power will decrease from *pmin* to 0 again (second RHS term)."""
        return model.p[g,t-1,n] - model.p[g,t,n] <= GenData.loc[g,'RampDown']*(1-model.u[g,t-1,n]+model.u[g,t,n]) +\
               GenData.loc[g,'pmin']*(2-model.u[g,t,n]-model.u[g,t-1,n])
    else:
        # since we initialized u with zero, so no ramp down rate needed
        return pyo.Constraint.Skip
model.RD = pyo.Constraint(G,T,N, rule = RD_rule)

## 7.5. Excess electricity limit & Curtailment limit <a class="anchor" id="Part_7_5"></a>

In [10]:
def hourly_excess_ele_ub_node_rule(model,t,n):
    """the amount of excess electricity at each hour should be less than the specified value for node n"""
    return model.EE[t,n] <= NodeData.loc[n,'hourly_excess_ele_ub']
model.hourly_excess_ele_ub_node = pyo.Constraint(T,N, rule = hourly_excess_ele_ub_node_rule)

def hourly_curtailed_ele_ub_node_rule(model,t,n):
    """the amount of curtailed electricity at each hour should be less than the specified value for node n"""
    return model.CE[t,n] <= NodeData.loc[n,'hourly_curtailed_ele_ub']
model.hourly_curtailed_ele_ub_node = pyo.Constraint(T,N, rule = hourly_curtailed_ele_ub_node_rule)

#################################################################################################################

def total_excess_ele_ub_node_rule(model,n):
    """total amount of excess elecctricity in node n should be less than the upper bound of that node"""
    coef = [1 for _ in range(len(T))]
    model.linexp = LinearExpression(constant=0,linear_coefs=coef,linear_vars=[model.EE[t,n] for t in T])
    return model.linexp <= NodeData.loc[n,'tot_excess_ele_ub']
    #return sum(model.EE[t,n] for t in T) <= NodeData.loc[n,'tot_excess_ele_ub']
model.total_excess_ele_ub_node = pyo.Constraint(N, rule = total_excess_ele_ub_node_rule)

def total_excess_ele_ub_rule(model):
    """total amount of excess electricity (in the optimization horizon) should be less than the specified value"""
    coef = [1 for _ in range(len(N)*len(T))]
    model.linexp2 = LinearExpression(constant=0,linear_coefs=coef,linear_vars=[model.EE[t,n] for t in T for n in N])
    return model.linexp2 <= Params.loc['total_excess_ele_ub','Val']
    #return sum(model.EE[t,n] for t in T for n in N) <= Params.loc['total_excess_ele_ub','Val']
model.total_excess_ele_ub = pyo.Constraint(rule = total_excess_ele_ub_rule)

#################################################################################################################

def total_curtailed_ele_ub_node_rule(model,n):
    """total amount of curtailed electricity in node n should be less than the upped bound of that node"""
    coef = [1 for _ in range(len(T))]
    model.linexp3 = LinearExpression(constant=0,linear_coefs=coef,linear_vars=[model.CE[t,n] for t in T])
    return model.linexp3 <= NodeData.loc[n,'tot_curtailed_ele_ub']
    #return sum(model.CE[t,n] for t in T) <= NodeData.loc[n,'tot_curtailed_ele_ub']
model.total_curtailed_ele_ub_node = pyo.Constraint(N, rule = total_curtailed_ele_ub_node_rule)

def total_curtailed_ele_ub_rule(model):
    """total amount of curtailed electricity (in the optimization horizon) should be less than the specified value"""
    coef = [1 for _ in range(len(T)*len(N))]
    model.linexp4 = LinearExpression(constant=0,linear_coefs=coef,linear_vars=[model.CE[t,n] for t in T for n in N])
    return model.linexp4 <= Params.loc['total_curtailed_ele_ub','Val']
    #return sum(model.CE[t,n] for t in T for n in N) <= Params.loc['total_curtailed_ele_ub','Val']
model.total_curtailed_ele_ub = pyo.Constraint(rule = total_curtailed_ele_ub_rule)

## 7.6. Balance (Demand satisfaction) <a class="anchor" id="Part_7_6"></a>

In [11]:
if Params.loc['consider_DemandResponse','Val'] == True:
    def demand_response_rule(model,n):
        coef = [1 for _ in range(len(T))]
        model.linexp5 = LinearExpression(constant=0,linear_coefs=coef,linear_vars=[model.D[t,n] for t in T])
        return model.linexp5 == sum( load.loc[t,n] for t in T )
        
# the below two lines will generate error as a constant (such as load.loc[t,n]) can not be in the LinearExpression function
        #model.linexp6 = LinearExpression(constant=0,linear_coefs=coef,linear_vars=[float(load.loc[t,n]) for t in T])
        #return model.linexp5 == model.linexp5

# traditional way to write this constraint which is slower than using LinearExpression
        #return sum( model.D[t,n] for t in T ) == sum( load.loc[t,n] for t in T )
    model.demand_response = pyo.Constraint(N, rule = demand_response_rule)


def thermal_generation_excess_rule(model,t,n):
    coef = [1 for _ in range(len(G))]
    model.linexp7 = LinearExpression(constant=0,linear_coefs=coef,linear_vars=[model.p[g,t,n] for g in G])
    return model.linexp7 == model.thermal_excess[t,n] + model.thermal_consumption[t,n]
    #return sum( model.p[g,t,n] for g in G ) == model.thermal_excess[t,n] + model.thermal_consumption[t,n]
model.thermal_generation_excess = pyo.Constraint(T,N, rule = thermal_generation_excess_rule)  


if Params.loc['consider_Renewable','Val'] == True:
    def renewable_excess_rule(model,t,n):
        return sum(RenNode.loc[n,r]*RenGen.loc[t,r] for r in R) == model.ren_excess[t,n]+model.ren_consumption[t,n]
    model.renewable_excess = pyo.Constraint(T,N, rule = renewable_excess_rule)


def excess_ele_rule(model,t,n):
    return model.EE[t,n] == model.ren_excess[t,n] + model.thermal_excess[t,n]
model.excess_ele = pyo.Constraint(T,N, rule = excess_ele_rule)
    
    
def demand_rule(model,t,n):
    """electricity demand or balance rule states that total input - total output = excess - curtailed
    total input = energy flow from other nodes to node n + storage discharge + generators' output in node n.
    total output = energy flow from node n to the others + storage charg + node demand(load)"""
    
    transaction   = sum( TransEff.loc[nn,n]*model.f[t,nn,n] - model.f[t,n,nn] for nn in N )
    
    if Params.loc['consider_Storage','Val'] == True:
        storage   = sum(model.SDIS[t,n,s] - model.SCH[t,n,s] for s in S)
    else:
        storage   = 0
    
    if Params.loc['consider_DemandResponse','Val'] == True:
        return model.thermal_consumption[t,n]+model.ren_consumption[t,n]+storage+transaction-model.D[t,n]+model.CE[t,n]==0
    else:
        return model.thermal_consumption[t,n]+model.ren_consumption[t,n]+storage+transaction-load.loc[t,n]+model.CE[t,n]==0
    
model.demand = pyo.Constraint(T,N, rule = demand_rule)

## 7.7. Sum of piece wise powers <a class="anchor" id="Part_7_7"></a>

In [12]:
def linear_power_rule(model,g,t,n):
    """total power generation of a generator is equal to sum of all linear powers fitted on that generator"""
    return model.p[g,t,n] == GenData.loc[g,'pmin']*model.u[g,t,n] + sum( model.pl[g,t,l,n] for l in L )
model.linear_power = pyo.Constraint(G,T,N, rule = linear_power_rule)

def linear_power_ub_rule(model,g,t,l,n):
    """at each linear part, PL must be less than the range in which line L in fitted on the cost-power curve"""
    return model.pl[g,t,l,n] <= linear_range.loc[g,'linear_range']*model.u[g,t,n]
model.linear_power_ub = pyo.Constraint(G,T,L,N, rule = linear_power_ub_rule)

## 7.8. Emission constraints <a class="anchor" id="Part_7_8"></a>

In [13]:
def linear_emission_rule(model,g,t,n):
    """total emission generated by gen g at time t in node n is equal to sum of all linear parts fitted
    on the emission-power curve. Also there is a constant emission which is activated when gen g is working."""
    fixed_emsn    = GenData.loc[g,'MinEmsn']*model.u[g,t,n]
    variable_emsn = sum( EmissionSlope.loc[g,l]*model.pl[g,t,l,n] for l in L )
    return model.emsn[g,t,n] == fixed_emsn + variable_emsn
model.linear_emission = pyo.Constraint(G,T,N, rule = linear_emission_rule)

def total_node_emission_ub(model,n):
    """total emission in node n must be less than an accepted value (in the horizon)"""
    coef = [1 for _ in range(len(G)*len(T))]
    model.linexp8 = LinearExpression(constant=0,linear_coefs=coef,linear_vars=[model.emsn[g,t,n] for g in G for t in T])
    return model.linexp8 <= NodeData.loc[n,'EmissionUb']
    #return sum( model.emsn[g,t,n] for g in G for t in T ) <= NodeData.loc[n,'EmissionUb']
model.total_node_emission = pyo.Constraint(N, rule = total_node_emission_ub)

def total_emission_ub_rule(model):
    """total emission considering all of nodes must be smaller than an accepted limit (in the horizon)"""
    coef = [1 for _ in range(len(G)*len(T)*len(N))]
    model.linexp9 = LinearExpression(constant=0,linear_coefs=coef,linear_vars=[model.emsn[g,t,n] for g in G for t in T for n in N])
    return model.linexp9 <= Params.loc['total_emission_ub','Val']
    #return sum( model.emsn[g,t,n] for g in G for t in T for n in N ) <= Params.loc['total_emission_ub','Val']
model.total_emission_ub = pyo.Constraint(rule = total_emission_ub_rule)

## 7.9. Transmission constraints <a class="anchor" id="Part_7_9"></a>

In [14]:
if Params.loc['consider_NetZeroEnergy','Val'] == True:   
    def net_zero_energy_rule(model,n):
        return sum( model.f[t,n,nn] for nn in N for t in T) >= NodeData.loc[n,'NetZeroEnCoef']*\
               sum( model.f[t,nn,n]*TransEff.loc[nn,n] for nn in N for t in T )
    model.net_zero_energy = pyo.Constraint(N,rule = net_zero_energy_rule)

def trans_ub_first_rule(model,t,n,nn):
    """energy flow between node n and nn must be smaller than the capacity of the transmission line."""
    return model.f[t,n,nn] <= TransUb.loc[n,nn]*model.fb[t,n,nn]
model.trans_ub_first = pyo.Constraint(T,N,N,rule = trans_ub_first_rule)

def trans_ub_second_rule(model,t,n,nn):
    """energy flow between node n and nn must be smaller than the capacity of the transmission line."""
    return model.f[t,nn,n] <= TransUb.loc[nn,n]*(1-model.fb[t,n,nn])
model.trans_ub_second = pyo.Constraint(T,N,N,rule = trans_ub_second_rule)

## 7.10. Storage Constraints <a class="anchor" id="Part_7_10"></a>

In [15]:
if Params.loc['consider_Storage','Val'] == True:

    def Storage_level_of_energy_rule(model,t,n,s):
        if t > 1:
            """storage level of energy (SLE) at each time depends on SLE at the previous time plus charging minus 
            discharging."""
            return model.SLE[t,n,s]==Storage.loc[s,'SelfDisFactor']*model.SLE[t-1,n,s]+\
                   Storage.loc[s,'ChEff']*model.SCH[t,n,s]-(1/Storage.loc[s,'DisEff'])*model.SDIS[t,n,s]-\
                   Storage.loc[s,'SelfDisConst']*StorNodes.loc[n,s]
        else:
            """at the beginning, instead of using SLEt-1, an initial value is used."""
            return model.SLE[t,n,s]==Storage.loc[s,'SelfDisFactor']*Storage.loc[s,'InitEnergy']*StorNodes.loc[n,s]+ \
                   Storage.loc[s,'ChEff']*model.SCH[t,n,s]-(1/Storage.loc[s,'DisEff'])*model.SDIS[t,n,s]- \
                   Storage.loc[s,'SelfDisConst']*StorNodes.loc[n,s]
    model.Storage_level_of_energy = pyo.Constraint(T,N,S, rule= Storage_level_of_energy_rule)

    #################################################################################################################

    def Storage_level_of_energy_ub_rule(model,t,n,s):
        """amount of energy stored in the storage s must be less than its capacity"""
        return model.SLE[t,n,s] <= Storage.loc[s,'SLEub']
    model.Storage_level_of_energy_ub = pyo.Constraint(T,N,S, rule= Storage_level_of_energy_ub_rule)

    def Storage_level_of_energy_lb_rule(model,t,n,s):
        """amount of energy stored in storage s must be above the lower limit"""
        return Storage.loc[s,'SLElb']*StorNodes.loc[n,s] <= model.SLE[t,n,s]
    model.Storage_level_of_energy_lb = pyo.Constraint(T,N,S, rule= Storage_level_of_energy_lb_rule)

    #################################################################################################################

    def Storage_charging_cap_ub_rule(model,t,n,s):
        """amount of energy that can enter the storage must be less than the charging upper bound"""
        return model.SCH[t,n,s] <= Storage.loc[s,'ChCap']*model.SCHB[t,n,s]
    model.Storage_charging_cap_ub = pyo.Constraint(T,N,S, rule= Storage_charging_cap_ub_rule)

    def Storage_discharging_cap_ub_rule(model,t,n,s):
        """amount of energy that can leave the storage must be less than the discharging upper bound"""
        return model.SDIS[t,n,s] <= Storage.loc[s,'DisCap']*model.SDISB[t,n,s]
    model.Storage_discharging_cap_ub = pyo.Constraint(T,N,S, rule= Storage_discharging_cap_ub_rule)
    
    def storage_charge_control_rule(model,t,n,s):
        return model.SDISB[t,n,s] + model.SCHB[t,n,s] <= 1
    model.storage_charge_control = pyo.Constraint(T,N,S, rule= storage_charge_control_rule)

## 7.11. N-1 Contingency <a class="anchor" id="Part_7_11"></a>

In [16]:
if Params.loc['consider_N-1Contingency','Val'] == True: 
    
    def gen_contingency_reserve_ub_first_rule(model,g,t,n):
        if GenNode.loc[n,g] == 1:
            return model.r[g,t,n] <= GenData.loc[g,'RampUp']*model.u[g,t,n] + GenData.loc[g,'pmin']*(1-model.u[g,t,n]
                                                                                                      -model.a[g,t,n])
        else:
            return pyo.Constraint.Skip
    model.gen_contingency_reserve_ub_first = pyo.Constraint(G,T,N, rule=gen_contingency_reserve_ub_first_rule)

    def gen_contingency_reserve_lb_rule(model,g,t,n):
        if GenNode.loc[n,g] == 1:
            return model.r[g,t,n] >= GenData.loc[g,'pmin']*(1-model.u[g,t,n]-model.a[g,t,n])
        else:
            return pyo.Constraint.Skip    
    model.gen_contingency_reserve_lb = pyo.Constraint(G,T,N, rule=gen_contingency_reserve_lb_rule)
    
    def gen_contingency_off_rule(model,g,t,n):
        if GenNode.loc[n,g] == 1:
            return model.a[g,t,n] <= 1 - model.u[g,t,n]
        else:
            return pyo.Constraint.Skip
    model.gen_contingency_off = pyo.Constraint(G,T,N, rule=gen_contingency_off_rule)
    
    def gen_contingency_reserve_ub_second_rule(model,g,t,n):
        if GenNode.loc[n,g] == 1:
            return model.r[g,t,n] <= GenData.loc[g,'pmax'] - model.p[g,t,n]
        else:
            return pyo.Constraint.Skip
    model.gen_contingency_reserve_ub_second = pyo.Constraint(G,T,N, rule=gen_contingency_reserve_ub_second_rule)
    
    #################################################################################################################

    def gen_contingency_lb_rule(model,g,t,n):
        if NodeData.loc[n,'N-1Contingency'] == True:
            if GenNode.loc[n,g] == 1:
                return sum(model.r[gg,t,nn] for gg in G for nn in N if gg!=g or nn!=n) >= model.p[g,t,n]
            else:
                return pyo.Constraint.Skip
        else:
            return pyo.Constraint.Skip
    model.gen_contingency_lb = pyo.Constraint(G,T,N, rule=gen_contingency_lb_rule)
    
    def gen_contingency_ub_rule(model,g,t,n):
        if NodeData.loc[n,'N-1Contingency'] == True:
            if GenNode.loc[n,g] == 1:
                return sum(model.r[gg,t,nn] for gg in G for nn in N if gg!=g or nn!=n) <=model.p[g,t,n]*(1+
                                                                                            Params.loc['reserve_ub','Val'])
            else:
                return pyo.Constraint.Skip
        else:
            return pyo.Constraint.Skip
    model.gen_contingency_ub = pyo.Constraint(G,T,N, rule=gen_contingency_ub_rule)

# 8. Solve the model <a class="anchor" id="Part_8"></a>

In [17]:
if Params.loc['solver_name','Val'] == 'cbc':
    solver_name = 'cbc.exe'
else:
    solver_name = Params.loc['solver_name','Val']

start_time = time.time()
results = pyo.SolverFactory(solver_name).solve(model)
end_time = time.time()

solver_status = results.solver.status
termination_condition = results.solver.termination_condition
run_time = end_time - start_time
if (solver_status == SolverStatus.ok) and (termination_condition == TerminationCondition.optimal):
    print(f"Solver: {Params.loc['solver_name','Val']}")
    print(f"Solver Status: OK")
    print("Termination Condition: Optimal")
    print(f"run time (s): {run_time}")
    pyo.assert_optimal_termination(results)
else:
    print(f"Solver: {Params.loc['solver_name','Val']}")
    print("Global optimal solution was not found.")
    print(f"Solver Status: {solver_status}")
    print(f"Termination Condition: {termination_condition}")

Solver: highs
Solver Status: OK
Termination Condition: Optimal
run time (s): 2.9640467166900635


## 8.1. Save the results in an excel file <a class="anchor" id="Part_8_1"></a>

In [18]:
def add_statistics(dataframe):
    dataframe.loc['max']  = dataframe.loc[T[0]:T[-1],:].max()
    dataframe.loc['min']  = dataframe.loc[T[0]:T[-1],:].min()
    dataframe.loc['sum']  = dataframe.loc[T[0]:T[-1],:].sum()
    dataframe.loc['mean'] = dataframe.loc[T[0]:T[-1],:].mean()
    dataframe.loc['std']  = dataframe.loc[T[0]:T[-1],:].std()
    dataframe.loc['non-zero min']  = dataframe.loc[T[0]:T[-1],:][dataframe.loc[T[0]:T[-1],:] > 0].min()
    dataframe.loc['non-zero mean'] = dataframe.loc[T[0]:T[-1],:][dataframe.loc[T[0]:T[-1],:] > 0].mean()
    dataframe.loc['non-zero std']  = dataframe.loc[T[0]:T[-1],:][dataframe.loc[T[0]:T[-1],:] > 0].std()
    return dataframe

In [19]:
# model_vars = model.component_map(ctype=pyo.Var)
# var_names = list(model_vars) # shows the list of defined variables
# index = list(model.p) # shows the index of every defined variable

power    = {}

if Params.loc['consider_N-1Contingency','Val'] == True:
    reserve  = {}

onoff    = {}
start    = {}
shutdown = {}
emission = {}

for n in N:
    for g in G:
        if GenNode.loc[n,g] == 1:
            power[(n,g)]    = {t: model.p[g,t,n].value for t in T}
            
            if Params.loc['consider_N-1Contingency','Val'] == True:
                reserve[(n,g)]  = {t: model.r[g,t,n].value for t in T}
                
            onoff[(n,g)]    = {t: round(model.u[g,t,n].value) for t in T}
            start[(n,g)]    = {t: round(model.v[g,t,n].value) for t in T}
            shutdown[(n,g)] = {t: round(model.w[g,t,n].value) for t in T}
            emission[(n,g)] = {t: model.emsn[g,t,n].value+round(model.v[g,t,n].value)*GenData.loc[g,'StartUpEmission']\
                               +round(model.w[g,t,n].value)*GenData.loc[g,'ShutDownEmission'] for t in T}

power = add_statistics(pd.DataFrame(power))

if Params.loc['consider_N-1Contingency','Val'] == True:
    reserve = add_statistics(pd.DataFrame(reserve))
    
emission = add_statistics(pd.DataFrame(emission))

#################################################################################################################

flow = {}

for n in N:
    for nn in N:
        if n!=nn:
            flow[(n,nn)] = {t: model.f[t,n,nn].value for t in T}
            
flow = add_statistics(pd.DataFrame(flow))

#################################################################################################################

if Params.loc['consider_NetZeroEnergy','Val'] == True: 
    netzeroenergy = pd.DataFrame(index = N, columns = ['flow out','flow in', 'flow in * NetZeroEnCoef',
                                                       'flow out / flow in', 'NetZeroEnCoef'])
    for n in N:
        netzeroenergy.loc[n,'flow out'] = sum( model.f[t,n,nn].value*TransEff.loc[n,nn] for nn in N for t in T )
        netzeroenergy.loc[n,'flow in']  = sum( model.f[t,nn,n].value*TransEff.loc[nn,n] for nn in N for t in T )
        netzeroenergy.loc[n,'flow in * NetZeroEnCoef']  = netzeroenergy.loc[n,'flow in']*NodeData.loc[n,'NetZeroEnCoef']
        netzeroenergy.loc[n,'flow out / flow in']  = netzeroenergy.loc[n,'flow out'] / netzeroenergy.loc[n,'flow in']
        netzeroenergy.loc[n,'NetZeroEnCoef'] = NodeData.loc[n,'NetZeroEnCoef']

#################################################################################################################

excess              = {}
curtailed           = {}

if Params.loc['consider_Renewable','Val'] == True:
    ren_excess          = {}
    ren_consumption     = {}

thermal_excess      = {}
thermal_consumption = {}

if Params.loc['consider_DemandResponse','Val'] == True:
    demand = {}
    
for n in N:
    excess[n]              = {t: model.EE[t,n].value for t in T}
    thermal_excess[n]      = {t: model.thermal_excess[t,n].value for t in T}
    
    if Params.loc['consider_Renewable','Val'] == True:
        ren_excess[n]          = {t: model.ren_excess[t,n].value for t in T}
        ren_consumption[n]     = {t: model.ren_consumption[t,n].value for t in T}
    
    thermal_consumption[n] = {t: model.thermal_consumption[t,n].value for t in T}
    
    curtailed[n]= {t: model.CE[t,n].value for t in T}
    
    if Params.loc['consider_DemandResponse','Val'] == True:
        demand[n]   = {t: model.D[t,n].value for t in T}
        
excess              = add_statistics(pd.DataFrame(excess))
thermal_excess      = add_statistics(pd.DataFrame(thermal_excess))

if Params.loc['consider_Renewable','Val'] == True:
    ren_excess          = add_statistics(pd.DataFrame(ren_excess))
    ren_consumption     = add_statistics(pd.DataFrame(ren_consumption))

thermal_consumption = add_statistics(pd.DataFrame(thermal_consumption))

curtailed           = add_statistics(pd.DataFrame(curtailed))


if Params.loc['consider_DemandResponse','Val'] == True:
    demand = add_statistics(pd.DataFrame(demand))

#################################################################################################################

if Params.loc['consider_Storage','Val'] == True:
    
    StorageLevelOfEnergy = {}
    StorageCharge = {}
    StorageDisCharge = {}
    
    for n in N:
        for s in S:
            if StorNodes.loc[n,s] == 1:
                StorageLevelOfEnergy[(n,s)] = {t: model.SLE[t,n,s].value for t in T}
                StorageCharge[(n,s)]        = {t: model.SCH[t,n,s].value for t in T}
                StorageDisCharge[(n,s)]     = {t: model.SDIS[t,n,s].value for t in T}  
                
    StorageLevelOfEnergy = add_statistics(pd.DataFrame(StorageLevelOfEnergy))
    StorageCharge = add_statistics(pd.DataFrame(StorageCharge))
    StorageDisCharge = add_statistics(pd.DataFrame(StorageDisCharge))

#################################################################################################################

if Params.loc['consider_Renewable','Val'] == True:
    Renewable_Generation = {}

    for n in N:
        for r in R:
            if RenNode.loc[n,r] == 1:
                Renewable_Generation[(n,r)] = {t: RenGen.loc[t,r] for t in T}

    Renewable_Generation = add_statistics(pd.DataFrame(Renewable_Generation))

#################################################################################################################

total_cost = model.SUSDC.value+model.OC.value+model.EEC.value+model.CEC.value+model.TC.value
total_income = model.RI.value+model.PI.value
net_economic_cost = total_cost - total_income
total_emission_cost = model.SUSDEC.value + model.OEC.value

Info = {'objective Value (considering weight factors)': pyo.value(model.obj),
        'start up and Shutdown cost'                  : model.SUSDC.value,
        'operation cost'                              : model.OC.value,
        'excess electricity cost'                     : model.EEC.value,
        'curtailed electricity cost'                  : model.CEC.value,
        'transmission cost'                           : model.TC.value,
        'total economic cost'                         : total_cost,
        'reserve income'                              : model.RI.value,
        'power income'                                : model.PI.value,
        'total income'                                : total_income,
        'net economic cost'                           : net_economic_cost,
        'weighted net economic cost'                  : Params.loc['cost_weight','Val']*net_economic_cost,
        'start up and Shutdown emission cost'         : model.SUSDEC.value,
        'operation emission cost'                     : model.OEC.value,
        'total emission cost'                         : total_emission_cost,
        'weighted emission cost'                      : Params.loc['emission_weight','Val']*total_emission_cost,
        'solver'                                      : Params.loc['solver_name','Val'],
        'run_time (s)'                                : run_time,
        'solver_status'                               : solver_status,
        'termination_condition'                       : termination_condition}

#################################################################################################################

with pd.ExcelWriter(address+"results.xlsx") as writer:
    
    power.to_excel(writer, sheet_name="power")
    
    if Params.loc['consider_N-1Contingency','Val'] == True:
        reserve.to_excel(writer, sheet_name="reserve")
        
    pd.DataFrame(onoff).to_excel(writer, sheet_name="onoff")
    pd.DataFrame(start).to_excel(writer, sheet_name="start")
    pd.DataFrame(shutdown).to_excel(writer, sheet_name="shutdown")
    
    emission.to_excel(writer, sheet_name="emission")
    
    flow.to_excel(writer, sheet_name="flow")
    
    if Params.loc['consider_NetZeroEnergy','Val'] == True:
        netzeroenergy.to_excel(writer, sheet_name="NetZero Energy")
    
    excess.to_excel(writer, sheet_name="excess")
    thermal_excess.to_excel(writer, sheet_name="thermal_excess")
    
    if Params.loc['consider_Renewable','Val'] == True:
        ren_excess.to_excel(writer, sheet_name="ren_excess")
        ren_consumption.to_excel(writer, sheet_name="ren_consumption")
    
    thermal_consumption.to_excel(writer, sheet_name="thermal_consumption")
    
    curtailed.to_excel(writer, sheet_name="curtailed")
    
    if Params.loc['consider_DemandResponse','Val'] == True:
        demand.to_excel(writer, sheet_name="demand")
    
    if Params.loc['consider_Renewable','Val'] == True:
        Renewable_Generation.to_excel(writer, sheet_name="Renewable Generation")
    
    if Params.loc['consider_Storage','Val'] == True:
        StorageLevelOfEnergy.to_excel(writer, sheet_name="StorageLevelOfEnergy")
        StorageCharge.to_excel(writer, sheet_name="StorageCharge")
        StorageDisCharge.to_excel(writer, sheet_name="StorageDisCharge")
    
    pd.DataFrame.from_dict(Info, orient='index', columns = ['Value']).to_excel(writer, sheet_name="OF & Info")

## 8.2. Plot results and add them to the excel file <a class="anchor" id="Part_8_2"></a>

In [ ]:
# https://www.geeksforgeeks.org/find-excel-column-name-given-number/

def excel_column_name(num):
    # num<= 16384
    alpha = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    if num < 26:
        return alpha[num-1]
    else:
        q, r = num//26, num % 26
        if r == 0:
            if q == 1:
                return alpha[r-1]
            else:
                return excel_column_name(q-1) + alpha[r-1]
        else:
            return excel_column_name(q) + alpha[r-1]

#################################################################################################################

def plot_result(result_path, result_dataframe, sheet_name, auxiliary_dataframe, set_, xlabel, ylabel, title):
    pic_name = 'chart.png'
    
    mode1_charts = ['power', 'emission', 'flow', 'Renewable Generation', 'StorageLevelOfEnergy', 
                    'StorageCharge', 'StorageDisCharge', 'reserve']
    
    wb = openpyxl.load_workbook(result_path)
    ws = wb[sheet_name]
    plt.figure(dpi=150, figsize=(5,len(N)*2));
    
    if sheet_name in mode1_charts:
        
        for n in N:
            InNode = [set_[i] for i in np.where(auxiliary_dataframe.loc[n,:] != 0)[0]]
            plt.subplot(len(N), 1, int(n[-1]));
            if len(InNode) >= 1:
                plt.plot(result_dataframe.loc[T[0]:T[-1],n], marker='o');
                plt.legend(InNode);
            else:
                plt.plot(np.zeros((T[-1],1)), marker='o');
                plt.legend(['Empty node']);
            plt.xlabel(xlabel);
            plt.ylabel(ylabel);
            plt.title(title + f' {n}');
            plt.grid();
        plt.tight_layout();
        
    else:
        
        for n in N:
            plt.subplot(len(N), 1, int(n[-1]));
            plt.plot(result_dataframe.loc[T[0]:T[-1],n], marker='o');
            plt.legend([n]);
            plt.xlabel(xlabel);
            plt.ylabel(ylabel);
            plt.title(title + f' {n}');
            plt.grid();
        plt.tight_layout();

    plt.savefig(pic_name);
    img = openpyxl.drawing.image.Image(pic_name);
    column, row = len(result_dataframe.columns)+2, 1
    img.anchor = excel_column_name(column) + f'{row}'
    ws.add_image(img);
    wb.save(result_path)
    os.remove(pic_name)

In [ ]:
if Params.loc['plot','Val'] == True:

    result_path = address+"results.xlsx"
    
    plot_result(result_path,power,    'power',   GenNode, G, 'Hour', 'MW',   'Power Generation in Node')
    
    if Params.loc['consider_N-1Contingency','Val'] == True:
        plot_result(result_path,reserve,  'reserve', GenNode, G, 'Hour', 'MW',   'Power Reserve in Node')
        
    plot_result(result_path,emission, 'emission',GenNode, G, 'Hour', 'Ton/h','Emission in Node')
    plot_result(result_path,flow,     'flow',    TransUb, N, 'Hour', 'MW',   'Transmission Line Node')
    plot_result(result_path,excess,   'excess',   0,      0, 'Hour', 'MW',   'Excess Electricity Generation in Node')
    plot_result(result_path,curtailed,'curtailed',0,      0, 'Hour', 'MW',   'Curtailed Electricity in Node')
    
    if Params.loc['consider_DemandResponse','Val'] == True:
        plot_result(result_path,demand,   'demand',   0,      0, 'Hour', 'MW',   'Demand in Node')
    
    if Params.loc['consider_Renewable','Val'] == True:
        plot_result(result_path,Renewable_Generation,'Renewable Generation',RenNode,  R,'Hour','MW',
                    'Renewable Generation in Node')
    
    if Params.loc['consider_Storage','Val'] == True:
        plot_result(result_path,StorageLevelOfEnergy,'StorageLevelOfEnergy',StorNodes,S,'Hour','MW',
                    'Storage level of energy in Node')
        plot_result(result_path,StorageCharge,       'StorageCharge',       StorNodes,S,'Hour','MW',
                    'Storage charging in Node')
        plot_result(result_path,StorageDisCharge,    'StorageDisCharge',    StorNodes,S,'Hour','MW',
                    'Storage discharging in Node')

# 9. To do list <a class="anchor" id="Part_9"></a>

to do list:
current v12
1- renewables (value of wind and solar curtailed, robust(best-worst-case), renewable incentive or curtail penalty, less than 24 hour and other cases [sec 4.1.3 power system modeling in GAMS])
2- generators start from different conditions and outputs (not only off), ends in a similar manner
3- add inverter model for solar
4- add nonlinear version with Bonmin
5- load factor as an objective function (average load / peak load->demand) see: [https://or.stackexchange.com/questions/711/how-to-formulate-linearize-a-maximum-function-in-a-constraint#:~:text=on%20this%20post.-,How%20to%20formulate%20(linearize)%20a%20maximum%20function%20in%20a%20constraint,and%20C%E2%A9%BEc2.] for max function linearization.
6- node-based electricity price
7- hourly demand response factors
10- consider loss and excess as an objective
11- consider reliability for renewable units as well
12- consider reliability based on demand or a percentage of it
13- consider using LinearExpression to speed up constraint creation time which will reduce model creation time [https://pyomo.readthedocs.io/en/stable/advanced_topics/linearexpression.html] also look at [PYOMO book section 9.2 page 129]
14- add lower bound to charging and discharging power of the storages
__________________________
1- CHP [springer briefs and power system modeling in GAMS sec 3.3]
2- hydro unit [3.4. power system modeling in GAMS]
3- multi-fuel, valve point effect
4- energy storage (power to gas and pumped hydro)
5- add more advanced transmission constraints [6. power system modeling in GAMS]
6- stochasticity (add all risk measures: variance, shortfall probability, expected shortage, value at risk, conditional value at risk) [pdf 12 and 13 in the applied optimization course]. stochasticity sources: renewables, load, line outage
7- chance constraint and robust
8- license [https://choosealicense.com/appendix/]
9- planning (add new generators, storages, renewables, transmission lines)
10- for more than 1 year, demand growth
11- water-energy nexus [10.1 power system modeling in GAMS]
12- for gas network see [Hub TA, Hub]
13- accurate unit commitment paper
14- consider transmission lines limitation in reserve constraint (ex 4C page 152 the UC book introduced by Dr. Moeini) 